In [1]:
import QuantLib as ql

In [2]:
euhicp = ql.EUHICP(True)

In [3]:
evaluationDate = ql.Date(13, ql.August, 2007)
evaluationDate = ql.UnitedKingdom().adjust(evaluationDate)
ql.Settings.instance().evaluationDate = evaluationDate

In [4]:
swap_type = ql.CPISwap.Payer

In [5]:
nominal = 1000000.0
subtractInflationNominal = True
spread = 0.0
floatDayCount = ql.Actual365Fixed()
floatPaymentConvention = ql.ModifiedFollowing

In [6]:
nomD = [
                ql.Date(26, ql.November, 2009),
                ql.Date(2, ql.December, 2009), 
                ql.Date(29, ql.December, 2009),
                ql.Date(25, ql.February, 2010),
                ql.Date(18, ql.March, 2010), 
                ql.Date(25, ql.May, 2010), 
                ql.Date(16, ql.September, 2010),
                ql.Date(16, ql.December, 2010),
                ql.Date(17, ql.March, 2011), 
                ql.Date(16, ql.June, 2011), 
                ql.Date(22, ql.September, 2011),
                ql.Date(25, ql.November, 2011),
                ql.Date(26, ql.November, 2012),
                ql.Date(25, ql.November, 2013),
                ql.Date(25, ql.November, 2014),
                ql.Date(25, ql.November, 2015),
                ql.Date(25, ql.November, 2016),
                ql.Date(27, ql.November, 2017),
                ql.Date(26, ql.November, 2018),
                ql.Date(25, ql.November, 2019),
                ql.Date(25, ql.November, 2021),
                ql.Date(25, ql.November, 2024),
                ql.Date(26, ql.November, 2029),
                ql.Date(27, ql.November, 2034),
                ql.Date(25, ql.November, 2039),
                ql.Date(25, ql.November, 2049),
                ql.Date(25, ql.November, 2059),
                ql.Date(25, ql.November, 2069),
                ql.Date(27, ql.November, 2079)
        ]

In [7]:
nomR = [
                    0.475    ,
                0.47498 ,
                0.49988 ,
                0.59955 ,
                0.65361 ,
                0.82830 ,
                0.78960 ,
                0.93762 ,
                1.12037 ,
                1.31308 ,
                1.52011 ,
                1.78399 ,
                2.41170 ,
                2.83935 ,
                3.12888 ,
                3.34298 ,
                3.50632 ,
                3.63666 ,
                3.74723 ,
                3.83988 ,
                4.00508 ,
                4.16042 ,
                4.15577 ,
                4.04933 ,
                3.95217 ,
                3.80932 ,
                3.80849 ,
                3.72677 ,
                3.63082 
]

In [8]:
nomR = [r/100. for r in nomR]

In [9]:
dcNominal = ql.ActualActual()

nominal = ql.ZeroCurve(nomD, nomR, dcNominal)

nominalTS = ql.RelinkableYieldTermStructureHandle()
nominalTS.linkTo(nominal)

In [10]:
fixingDays = 0



floatIndex = ql.GBPLibor(ql.Period("6M"), nominalTS)

In [11]:
fixedRate = 0.1
baseCPI = 206.1
fixedDayCount = ql.Actual365Fixed()

In [12]:
fixedPaymentConvention = ql.ModifiedFollowing
fixedPaymentCalendar = ql.UnitedKingdom()

In [13]:
hcpi = ql.RelinkableZeroInflationTermStructureHandle()
fixedIndex = ql.UKRPI(False, hcpi)
calendar = ql.UnitedKingdom()
dcZCIIS = ql.ActualActual()
observationLag = ql.Period(2,ql.Months)
baseZeroRate = 3.0495 / 100

helpers = [ql.ZeroCouponInflationSwapHelper(0.01, ql.Period("2M"), evaluationDate+ql.Period("1Y"), calendar, ql.ModifiedFollowing, dcZCIIS, fixedIndex)]
pCPIts = ql.PiecewiseZeroInflation(evaluationDate, calendar, dcZCIIS, observationLag, fixedIndex.frequency(),
                                  fixedIndex.interpolated(), baseZeroRate, ql.YieldTermStructureHandle(ql.FlatForward(evaluationDate,0.03,ql.Actual365Fixed())),
                                  helpers)
pCPIts.enableExtrapolation()
hcpi.linkTo(pCPIts)


In [14]:
fixedIndex = ql.UKRPI(False, hcpi)

In [15]:
stype = ql.CPISwap.Payer
nominal = 1000000.0
inflationNominal = nominal
quote = 0.03714
floatNominal = inflationNominal * (1.0+quote)**50
dummySpread = 0.0
dummyFixedRate = 0.0
dummyDC = ql.ActualActual()
endDate = ql.Date(25, ql.November, 2059)
oneDate = [endDate]
cal = ql.UnitedKingdom()
paymentConvention = ql.ModifiedFollowing
schOneDate = ql.Schedule(oneDate, cal, paymentConvention)
dummyFloatIndex = ql.Euribor6M(ql.YieldTermStructureHandle(ql.FlatForward(evaluationDate,0.03,ql.Actual365Fixed(), True)))

In [16]:
cS = ql.CPISwap(stype, floatNominal, subtractInflationNominal, dummySpread, dummyDC, schOneDate,
               paymentConvention, fixingDays, dummyFloatIndex,
               dummyFixedRate, baseCPI, dummyDC, schOneDate, paymentConvention, observationLag,
               fixedIndex, ql.CPI.AsIndex, inflationNominal)

In [17]:
dse = ql.DiscountingSwapEngine(nominalTS)

In [18]:
cS.setPricingEngine(dse)

In [19]:
fixedIndex.clearFixings()
fixedIndex.addFixing(ql.Date(1,6,2007), 1.0)

In [20]:
cS.NPV()

nan